In [1]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
import altair as alt
import time
import itertools
import numpy as np
import operator
from datetime import timedelta
from ipywidgets import interact, IntSlider
from IPython.display import display
import sklearn
import numpy as np
from sklearn.linear_model import LogisticRegression
# import seaborn as sns
# from matplotlib import pyplot as plt
# %matplotlib inline
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from imblearn.under_sampling import RandomUnderSampler

from pycaret.classification import * 

# pd.options.display.max_columns = 5000
pd.options.display.max_rows = 5000

injured_pred = 'injured_in_26_week'

simple_features = ['Height', 'Weight', 'age','cum_injury_total', 'weeks_since_last_injury', 'Min_cum','Serie A_cum',
 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum',
 'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum']

extended_features = ['Height', 'Weight', 'defender', 'attacker', 'midfielder', 'goalkeeper', 'right_foot', 'age', 'cum_injury_total', 'weeks_since_last_injury', 'Min_cum', 'Gls_cum', 'Ast_cum', 'PK_cum', 'PKatt_cum',
 'Sh_cum', 'SoT_cum', 'CrdY_cum', 'CrdR_cum', 'Touches_cum', 'Press_cum', 'Tkl_cum', 'Int_cum', 'Blocks_cum', 'xG_cum', 'npxG_cum', 'xA_cum', 'SCA_cum', 'GCA_cum', 'Cmp_cum',
 'Att_cum', 'Prog_cum', 'Carries_cum', 'Prog.1_cum', 'Succ_cum', 'Att.1_cum', 'Fls_cum', 'Fld_cum', 'Off_cum', 'Crs_cum', 'TklW_cum', 'OG_cum', 'PKwon_cum','PKcon_cum', 'Serie A_cum',
 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', 'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum',
 'Copa América_cum', 'Away_cum', 'Home_cum', 'Neutral_cum']

/usr/local/lib/python3.6/dist-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
# Load Dataset

# Renzo's Path
# dataset = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/complete_final_df_5.csv')
#
# Sravan's Path
# dataset = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/dataset_for_model_final.csv')

# Sravan's Parquet Path
dataset = pd.read_parquet('dataframes_blog/dataset_for_model_final.parquet') 

In [3]:
# dataset[dataset['FBRefID']=='dea698d9'].to_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/ronaldo.csv', index=False)
dataset = dataset[dataset['drop'] == 0]
dataset = dataset[dataset['goalkeeper'] == 0]

In [4]:
# Get Train Test Split
df_train = dataset[dataset['cum_week'] <= dataset["train_split"]].dropna()
df_test = dataset[dataset['cum_week'] > dataset["train_split"]].dropna()

In [5]:
X_train = df_train[extended_features]
y_train = df_train[injured_pred]

X_test = df_test[extended_features]
y_test = df_test[injured_pred]

In [6]:
exp_clf = setup(dataset[extended_features + [injured_pred]], target=injured_pred, fix_imbalance=True, feature_selection=True, fold=2, fold_strategy='timeseries')

,Description,Value
0,session_id,6418
1,Target,injured_in_26_week
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(1604775, 59)"
5,Missing Values,True
6,Numeric Features,53
7,Categorical Features,5
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
set_config('X_train', X_train)
set_config('X_test', X_test)
set_config('y_train', y_train)
set_config('y_test', y_test)

In [8]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.9280,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,4.2150
rf,Random Forest Classifier,0.9123,0.5752,0.0288,0.1029,0.0447,0.0143,0.0174,23.2900
lightgbm,Light Gradient Boosting Machine,0.8989,0.5953,0.0635,0.1261,0.0812,0.0349,0.0383,20.0750
et,Extra Trees Classifier,0.8860,0.5732,0.0678,0.0942,0.0789,0.0198,0.0201,23.9100
ridge,Ridge Classifier,0.8336,0.0000,0.2229,0.1269,0.1617,0.0771,0.0808,4.9850
lda,Linear Discriminant Analysis,0.8336,0.6082,0.2229,0.1269,0.1617,0.0771,0.0808,12.0150
gbc,Gradient Boosting Classifier,0.8295,0.6390,0.2416,0.1308,0.1696,0.0841,0.0890,410.8750
dt,Decision Tree Classifier,0.7791,0.5135,0.2044,0.0826,0.1175,0.0168,0.0190,27.0850
ada,Ada Boost Classifier,0.7407,0.6246,0.3896,0.1153,0.1779,0.0752,0.0950,105.1800
knn,K Neighbors Classifier,0.7231,0.5386,0.2950,0.0858,0.1330,0.0241,0.0307,305.1450


In [10]:
# df = pull()
# df.to_csv('results_1_year.csv', index=False)
df = pull().sort_values(by=['F1'], ascending=False)

In [ ]:
# Select best model
model = create_model(df.index[0], fold=10)
save_model(model, 'model_1_semester')

IntProgress(value=0, description='Processing: ', max=4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:31:10
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,Ada Boost Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


In [ ]:
tuned_model = tune_model(model, optimize = 'F1')
# save_model(tuned_model, 'model_1_semester_tuned')

In [ ]:
clf = tuned_model.set_params(class_weight='balanced')
save_model(clf, 'model_1_quarter_tuned')

In [ ]:
# Loading the saved model
# model_1_semester_tuned = load_model('model_1_semester_tuned')

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_train, clf.predict(X_train), average='macro'))

print(recall_score(y_test, y_pred, average='macro'))
print(recall_score(y_train, clf.predict(X_train), average='macro'))

print(precision_score(y_test, y_pred, average='macro'))
print(precision_score(y_train, clf.predict(X_train), average='macro'))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')

In [ ]:
interpret_model(clf)

In [ ]:
plot_model(clf,'confusion_matrix')

In [ ]:
plot_model(clf,'pr')

In [ ]:
plot_model(clf,'feature')

In [ ]:
plot_model(clf,'parameter')

In [ ]:
plot_model(clf,'vc')

In [ ]:
plot_model(clf,'calibration')

In [ ]:
plot_model(clf,'learning')

In [ ]:
plot_model(clf,'class_report')

In [ ]:
plot_model(clf,'error')

In [ ]:
plot_model(clf, plot='ks')

In [ ]:
plot_model(clf, plot='threshold')

In [3]:
model_1_semester_tuned[23]

AdaBoostClassifier(learning_rate=0.3, n_estimators=140, random_state=8301)

In [10]:
# Cleaning NaNs

dataset_no_nans = dataset.fillna(0)

In [11]:
# Choosing the features we are going to use

dataset_to_predict = dataset_no_nans[['Height', 'Weight', 'defender', 'attacker', 'midfielder', 'goalkeeper', 'right_foot', 'age', 'cum_injury_total', \
    'weeks_since_last_injury', 'Min_cum', 'Gls_cum', 'Ast_cum', 'PK_cum', 'PKatt_cum', 'Sh_cum', 'SoT_cum', 'CrdY_cum', 'CrdR_cum', \
    'Touches_cum', 'Press_cum', 'Tkl_cum', 'Int_cum', 'Blocks_cum', 'xG_cum', 'npxG_cum', 'xA_cum', 'SCA_cum', 'GCA_cum', 'Cmp_cum', 'Att_cum', \
    'Prog_cum', 'Carries_cum', 'Prog.1_cum', 'Succ_cum', 'Att.1_cum', 'Fls_cum', 'Fld_cum', 'Off_cum', 'Crs_cum', 'TklW_cum', 'OG_cum', \
    'PKwon_cum', 'PKcon_cum', 'Serie A_cum', 'Premier League_cum', 'La Liga_cum', 'Ligue 1_cum', 'Bundesliga_cum', 'Champions Lg_cum', \
    'Europa Lg_cum', 'FIFA World Cup_cum', 'UEFA Nations League_cum', 'UEFA Euro_cum', 'Copa América_cum', 'Away_cum', 'Home_cum', \
    'Neutral_cum']]

In [12]:
#Using saved model to predict injuries for 1 year

predicted_injuries = model_1_semester_tuned[23].predict(dataset_to_predict)

In [13]:
# This is to predict the probability - WE ARE NOT USING IT
# prob_of_injury = model_1_semester_tuned[23].predict_proba(dataset_to_predict)[:, 1]

In [14]:
# Assigning the complet y target to y_target variable

y_target = list(y_train).append(list(y_test))

In [15]:
# Creating predictions_df to include the whole dataset plus predicted injuries and actual injures

predictions_df = dataset_to_predict
predictions_df['predicted_injuries'] = predicted_injuries
predictions_df['actual_injuries'] = y_target

In [16]:
# Merging the complete dataset with the predictions of the model

dataset_with_prediction = pd.merge(dataset, predictions_df['predicted_injuries'], left_index=True, right_index=True)


In [17]:
# Picking the columns we are going to keep

dataset_with_prediction = dataset_with_prediction[['FBRefID', 'name', 'date', 'Min', 'Position:', 'cum_week', 'defender', 'attacker', 'midfielder', 'goalkeeper', \
    'cum_injury_in_1_week', 'cum_injury_in_4_week',  'cum_injury_in_12_week', 'cum_injury_in_26_week', 'cum_injury_in_52_week', 'Min_cum', \
    'predicted_injuries']]

In [18]:
# Creating unique_predicted_injuries to filter out repeated injuries

dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries'] == 1) & (dataset_with_prediction['predicted_injuries']. \
    shift(1) == 1), 'unique_predicted_injuries'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries'] == 1) & (dataset_with_prediction['predicted_injuries']. \
    shift(1) == 0), 'unique_predicted_injuries'] = 1
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries'] == 0) & (dataset_with_prediction['predicted_injuries']. \
    shift(1) == 0), 'unique_predicted_injuries'] = 0
dataset_with_prediction.loc[(dataset_with_prediction['predicted_injuries'] == 0) & (dataset_with_prediction['predicted_injuries']. \
    shift(1) == 1), 'unique_predicted_injuries'] = 0

In [19]:
# Getting the accumulated sum of predicted injuries by player

dataset_with_prediction['cum_predicted_injuries'] = dataset_with_prediction.groupby(by=['name'])['unique_predicted_injuries'].cumsum()

In [69]:
# Picking player to forecast

picked_player_df = dataset_with_prediction[dataset_with_prediction['name'] == 'Neymar']

In [70]:
# Picking the features we are going to use

picked_player_df = picked_player_df[['FBRefID', 'name', 'date', 'Min', 'Position:', 'cum_week', 'defender','attacker', 'midfielder', \
    'goalkeeper', 'cum_injury_in_1_week', 'cum_injury_in_4_week', 'cum_injury_in_12_week', 'cum_injury_in_26_week', 'cum_injury_in_52_week', \
    'Min_cum', 'unique_predicted_injuries']]

In [71]:
# Function to Forecast players injuries and plot a chart

def forecast_it(weeks, df):
    df.reset_index(inplace=True)
    final_index = df.index.max()
    pos = 'cum_injury_in_'+str(weeks)+'_week'
    for n in range(weeks,0,-1): 
        df.loc[final_index-n+1, pos] = ''
    df.loc[df[pos] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = df[pos].shift(1) 
    df.loc[df['single_injury_prediction_in_'+str(weeks)+'_week'] == "", 'single_injury_prediction_in_'+str(weeks)+'_week'] = \
        df['unique_predicted_injuries']
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['single_injury_prediction_in_'+str(weeks)+'_week'].cumsum()
    df['cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_prediction_in_'+str(weeks)+'_week'].fillna('')
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'cum_injury_prediction_in_'+str(weeks)+'_week'] = df['cum_injury_in_'+str(weeks)+'_week']
    
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] != "", 'type_for_'+str(weeks)] = 'Actual Injuries'
    df.loc[df['cum_injury_in_'+str(weeks)+'_week'] == "", 'type_for_'+str(weeks)] = 'Predicted Injuries'

    extra_point = df[df['type_for_'+str(weeks)] == 'Actual Injuries'].tail(1)
    extra_point['type_for_'+str(weeks)] = 'Predicted Injuries'
    df = pd.concat([df, extra_point])

    df_final = df[len(df)-85:].sort_index()

    domain_min = df_final['cum_injury_prediction_in_'+str(weeks)+'_week'].min()-5
    domain_max = df_final['cum_injury_prediction_in_'+str(weeks)+'_week'].max()+5

    chart = alt.Chart(df_final).mark_line().encode(x=alt.X('date', timeUnit='yearmonthdate', axis=alt.Axis(tickCount=4)), \
        y=alt.Y('cum_injury_prediction_in_'+str(weeks)+'_week', \
        scale=alt.Scale(domain=[domain_min, domain_max])), color=alt.Color('type_for_'+str(weeks)), strokeDash=alt.condition(
        alt.datum.symbol == 'type_for_'+str(weeks), alt.value([5, 5]), alt.value([0]))). \
        properties(title = str(df['name'].iloc[0])+"'s Injury Prediction for "+str(weeks)+" weeks", width=800, height=300)
    
    return chart


In [72]:
#Forecasting 1 Semester(26 Weeks)

result_26_weeks = forecast_it(26, picked_player_df)

In [73]:
result_26_weeks

alt.Chart(...)